In [ ]:
import os
import gradio as gr
import requests
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API")
if api_key and api_key[:8] == "sk-proj-":
    print("API KEY IS READY")
else:
    print("Something went wrong")

In [ ]:
openai = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

In [ ]:
system_prompt = """
You are a helpful assistant please answer to the user's query precisely.
If you don't know the answer just say that I am not familiar with it.
"""

In [ ]:
def message_gpt(prompt):
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":prompt}
    ]
    completion = openai.chat.completions.create(
        model = MODEL,
        messages=messages,
    )
    response = completion.choices[0].message.content
    return response

In [ ]:
message_gpt("What is todays date")

In [ ]:
def UP_UP(text):
    return text.upper()

In [ ]:
gr.Interface(fn=UP_UP,inputs="textbox",outputs="textbox",allow_flagging="never").launch()
# gr.Interface(fn=UP_UP,inputs="textbox",outputs="textbox",allow_flagging="never").launch(share=True)

In [ ]:
view = gr.Interface(
    fn=message_gpt,
    inputs=gr.Textbox(label="Your Message",lines=6),
    outputs=gr.Textbox(label="Response",lines=8),
    allow_flagging="never"
)
view.launch()

In [ ]:
view = gr.Interface(
    fn=UP_UP,
    inputs=gr.Textbox(label="Your Message",lines=6),
    outputs=gr.Markdown(label="Response"),
    allow_flagging="never"
)
view.launch()

In [ ]:
def stream_output(prompt):
    messages = [
        {"role":"system","content":system_prompt},
        {"role":"user","content":prompt}
    ]
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_output,
    inputs=gr.Textbox(label="Your Message:"),
    outputs=gr.Markdown(label="Response"),
    allow_flagging="never",
)
view.launch()

In [ ]:
class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(self.url)
        self.body = response.content
        soup = BeautifulSoup(self.body,"html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator='\n',strip=True)
    def get_contents(self):
        return f"Webpage Title: {self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
system_prompt = "You are an assitant that analyzes the contents of a company landing page \
and create a short brochure about the company for customers, investors and recruiters. Respond in Markdown"

In [ ]:
def stream_brochure(title,url):
    prompt = f"Please provide a company brochure for {title}, here is their landing page:\n{Website(url).get_contents()}"
    result = stream_output(prompt=prompt)
    for chunk in result:
        yield chunk

In [ ]:
# stream_brochure("Anthropic","https://anthropic.com")

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs= [gr.Textbox(label="Company name: "),
             gr.Textbox(label="Landing Page URL: "),
             ],
    outputs=gr.Markdown(label="Brochure:"),
    allow_flagging="never",
)
view.launch()